In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import os

Using TensorFlow backend.


In [0]:
batch_size = 32
num_classes = 10
epochs = 10

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [0]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 25s 507us/step - loss: 1.8669 - acc: 0.3284 - val_loss: 1.4338 - val_acc: 0.4854
Epoch 2/10
50000/50000 [==============================] - 23s 470us/step - loss: 1.4753 - acc: 0.4678 - val_loss: 1.3115 - val_acc: 0.5311
Epoch 3/10
50000/50000 [==============================] - 24s 474us/step - loss: 1.3260 - acc: 0.5267 - val_loss: 1.5099 - val_acc: 0.4835
Epoch 4/10
50000/50000 [==============================] - 24s 475us/step - loss: 1.2088 - acc: 0.5711 - val_loss: 1.0856 - val_acc: 0.6231
Epoch 5/10
50000/50000 [==============================] - 24s 475us/step - loss: 1.1179 - acc: 0.6045 - val_loss: 1.0171 - val_acc: 0.6416
Epoch 6/10
50000/50000 [==============================] - 24s 476us/step - loss: 1.0486 - acc: 0.6307 - val_loss: 1.0629 - val_acc: 0.6271
Epoch 7/10
50000/50000 [==============================] - 24s 476us/step - loss: 0.9849 - acc: 0.6551 - val_loss: 0.

Experiments:

0. No BatchNormalization layer at all (by default). 

Hyperparameters: 10 epochs, optimizer - RMSProp, learning_rate=0.0001, loss - categorical_cross_entropy, batch_size=32, architecture - as it is now. Test set accuracy: **68%**

1.   Add BatchNormalization layer everywhere before activation.

Observation: causes test accuracy reduction because of overfitting. Test set accuracy: **11%**

2.   Remove BatchNormalization from the last Dense layer, leave in convolutional part of the network. 

Observation: test set accuracy grows a lot in comparison to previous experiment. Test set accuracy: **71%**

3. Add BatchNormalization layers only at the first block of convolution (before first dropout, 2 BatchNorms).

Observation: the network behaves pretty much the same as in previous experiment. Test set accuracy: **71%**

4. Add BatchNormalization layers only at the second block of convolution (2 BatchNorms).

Observation: Behaves better than any previous network architectures. Test set accuracy: **73%**


SOTA: **73%**
